## Cassandra database Model for Analyzing Songs And User Listening Activities

The event data, collected by Sparkify Music Streaming app, is comprised of CSV files for one day per file, stored in /data/event_data/ directory.

This project design a NoSQL database using Apache Cassandra to analyze the data collected on songs and user activity on by Sparkify''s new music streaming app. This database is designed to allow the analysis team to query the data to understand what songs users are listening to. 

This notebook includes two major components: 
I. ETL pipeline for pre-processing the event files into csv file for importing into the database, and
II. Using the processed csv data file, design and create tables suitable for running specific queries

### Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [37]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [38]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/data/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    

/Users/ai.tran/src/data-engineering/data-modeling/etl-pipeline


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [39]:
# specify the name of the file in which the extracted data is stored
processed_events_file = os.getcwd() + '/data/processed_events/event_datafile_new.csv'

# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    #print(f)
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open(processed_events_file, 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [40]:
# check the number of rows in your csv file
#with open(processed_events_file, 'r', encoding = 'utf8') as f:
#    print(sum(1 for line in f))

df = pd.read_csv(processed_events_file, header=0)
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26
2,Train,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Marry Me,26
3,Sony Wonder,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",597,Blackbird,61
4,Van Halen,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",602,Best Of Both Worlds (Remastered Album Version),80


In [41]:
df.shape

(6820, 11)

### Part II. Complete the Apache Cassandra coding portion of your project. 

#### Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="assets/image_event_datafile_new.jpg">

#### Creating a Cluster

In [42]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
try:
    from cassandra.cluster import Cluster
    cluster = Cluster(['127.0.0.1'], 6000)

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [43]:
# Create a Keyspace
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity
        WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [44]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [45]:
def create_table(session, table_name, table_columns, primary_keys, clustering_keys=None):
    """ 
        Create a table in current name space from given statement.
        This includes running `drop table` operation first to ensure 
        existing table is deleted prior to creating a new one.
    """

    if session is None or table_name is None or table_columns is None or primary_keys is None or len(primary_keys) == 0:
        print("Must provide active session, table name and columns plus data type, and one or more primary keys")
        return

    ## drop existing table with same name
    drop_table(session, table_name)
        
    try:
        ## declare table
        create_stmt = f"CREATE TABLE IF NOT EXISTS {table_name} ("

        ## define columns in table with their corresponding data type
        for column_name, data_type in table_columns:
            create_stmt = create_stmt + f"{column_name} {data_type}, "

        ## define primary keys for table
        create_stmt = create_stmt + f"PRIMARY KEY (({', '.join(primary_keys)})"

        ## append clustering keys for sorting if given
        if clustering_keys is not None:
            create_stmt = create_stmt + f", {', '.join(clustering_keys)}"
        create_stmt = create_stmt + "))"

        print(create_stmt)
        ## execute create operation
        session.execute(create_stmt)
    except Exception as e:
        print(e)


def populate_table(session, table_name, table_columns):
    """ Populate the table with data extracted from given data file """

    if session is None or table_name is None or table_columns is None:
        print("Must provide active session, table name plus columns plus data type")
        return

    insert_stmt = f"INSERT INTO {table_name} ({', '.join(table_columns)}) VALUES ({'%s, '*(len(table_columns)-1)}%s)"
    try:
        #df = pd.read_csv(datafile, header=0)
        [session.execute(insert_stmt, tuple(row)) for row in df[table_columns].to_numpy()]
    except Exception as e:
        print(e)


def fetch(session, query):
    """ Fetch data from database using given query statement """

    if session is None or query is None:
        print("Must provide active session, and query to execute")
        return

    try:
        rows = session.execute(query)
        print(f"===== Query: =======\n{query}\n")
        print( "===== Results: =====")
        [print(row) for row in rows]
    except Exception as e:
        print(e)
    

def drop_table(session, table_name):
    """ Drop the given table from keyspace of given session """

    drop_stmt = f"DROP TABLE IF EXISTS {table_name}"
    try:
        session.execute(drop_stmt)
    except Exception as e:
        print(e)

### Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [46]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
table_name = "music_app_history"

## create a table for storing data
create_table(session, 
    table_name=table_name, 
    table_columns=[('sessionId','int'), ('itemInSession','int'), ('artist','text'), ('song','text'), ('length','float')], 
    primary_keys=('sessionId', 'itemInSession')
)

## populate the table with data extracted from given data file
populate_table(session, table_name, ['sessionId', 'itemInSession', 'artist', 'song', 'length'])

## Add in the SELECT statement to verify the data was entered into the table
query = f"""
    SELECT artist, song, length
    FROM {table_name}
    WHERE sessionId=338 AND itemInSession=4"""
fetch(session, query)

CREATE TABLE IF NOT EXISTS music_app_history (sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY ((sessionId, itemInSession)))
===== Query: =======

    SELECT artist, song, length
    FROM music_app_history
    WHERE sessionId=338 AND itemInSession=4

===== Results: =====
Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


In [47]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
create_table(session, 
    table_name=table_name, 
    table_columns=[('userId','int'), ('sessionId','int'), ('itemInSession','int'), ('artist','text'), ('song','text'), ('firstName','text'), ('lastName','text')], 
    primary_keys=['userId', 'sessionId'],
    clustering_keys=['itemInSession']
)

populate_table(session, table_name, ['userId', 'sessionId', 'itemInSession', 'artist', 'song', 'firstName', 'lastName'])

query = f"""
    SELECT artist, song, firstName, lastName
    FROM {table_name}
    WHERE userId=10 AND sessionId=182 
    ORDER BY itemInSession"""
fetch(session, query)

CREATE TABLE IF NOT EXISTS music_app_history (userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))
===== Query: =======

    SELECT artist, song, firstName, lastName
    FROM music_app_history
    WHERE userId=10 AND sessionId=182 
    ORDER BY itemInSession

===== Results: =====
Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [48]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
create_table(session, 
    table_name=table_name, 
    table_columns=[('song','text'), ('userId','int'), ('firstName','text'), ('lastName','text')], 
    primary_keys=['song'],
    clustering_keys=['userId']
)

populate_table(session, table_name, ['song', 'userId', 'firstName', 'lastName'])

query = f"""
    SELECT firstName, lastName 
    FROM {table_name} 
    WHERE song='All Hands Against His Own'"""
fetch(session, query)               

CREATE TABLE IF NOT EXISTS music_app_history (song text, userId int, firstName text, lastName text, PRIMARY KEY ((song), userId))
===== Query: =======

    SELECT firstName, lastName 
    FROM music_app_history 
    WHERE song='All Hands Against His Own'

===== Results: =====
Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


#### Drop the tables before closing out the sessions

In [49]:
## Drop the table before closing out the sessions
drop_table(session, table_name)


#### Close the session and cluster connection¶

In [50]:
session.shutdown()
cluster.shutdown()